In [70]:
from google.oauth2.service_account import Credentials
from airflow.operators.python import PythonOperator
from google.cloud.storage.client import Client as StorageClient
from google.cloud.bigquery import Client as BqClient
from airflow.decorators import task, dag
from datetime import timedelta
from io import BytesIO

import pandas as pd
import datetime
import pendulum
import airflow
import logging
import os

In [11]:
LOG_FORMAT = "%(asctime)s [%(levelname)s]: %(threadName)s - %(message)s"
logging.basicConfig(format=LOG_FORMAT)
logger = logging.getLogger(__name__)
logger.setLevel("INFO")

# Constantes

CREDENTIALS_PATH = "/home/costa/Documentos/KarHub/karhub-challenge/airflow/credentials/karhub-key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = CREDENTIALS_PATH

DAG_ID = "processing_dag"
DAG_OWNER_NAME = "Data Engineering"
SCHEDULE_INTERVAL = None  # Dag com agendamento MANUAL
BUCKET = "etl-karhub-dados-sp"
START_DATE = pendulum.today('UTC').add(days=-1)
PROJECT = "karhub-434807"

In [7]:

def csv_to_storage(
    local_file_path: str, gcs_file_path: str, bucket_name: str = "etl-karhub-dados-sp"
):
    client = StorageClient(project=PROJECT)
    try:
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(gcs_file_path)
        blob.upload_from_filename(local_file_path)
        logger.info("Upload do arquivo %s foi realizado", local_file_path)
    except Exception as e:
        logger.error("Erro encontrado: %s", e)

In [9]:
def storage_to_raw(
    gcs_file_path: str,
    dataset_name: str,
    table_name: str,
    bucket_name: str = "etl-karhub-dados-sp",
):
    client = StorageClient(project=PROJECT)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(gcs_file_path)

In [26]:
client = StorageClient(project=PROJECT)
bucket = client.bucket("etl-karhub-dados-sp")
blob_receitas = bucket.blob("receitas/gdvReceitas.csv")
blob_despesas = bucket.blob("despesas/gdvDespesas.csv")

In [81]:
content = blob_receitas.download_as_bytes()

In [87]:
pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows', 9999)

data = BytesIO(content)
df = pd.read_csv(data, encoding="latin_1")

In [88]:
df

,Fonte de Recursos,Receita,Arrecadado
0,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11130111 - IRRF - IMP.RENDA PESSOA FISICA-PRINCIPAL,"4267858335,83"
1,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11130311 - IRRF-IMP. RENDA RET. FTE-TRABALHO-PRINCIPAL,"4323772997,93"
2,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11130341 - IRRF-OUTROS RENDIMENTOS-PRINCIPAL,"123071158,14"
3,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11180121 - IPVA - IMP. PROPR. VEIC. AUTOM - PRINCIPAL,"8202995933,47"
4,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11180122 - IPVA - MULTAS E JUROS,"6927,39"
5,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11180123 - IPVA - DIVIDA ATIVA,"235614546,72"
6,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11180124 - IPVA - MULTA E JUROS-DIVIDA ATIVA,"401195,68"
7,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11180131 - ITCMD-IMP.TRA.CAUSA MORTIS/DOACAO-PRINCIPAL,"3158253957,69"
8,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11180132 - ITCMD - MULTAS E JUROS,"218862721,73"
9,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,11180133 - ITCMD - DIVIDA ATIVA,"8067099,83"


## Problemas

Ao ler esse dataframe, foi criada uma coluna chamada "Unnamed: 3", proveniente da coluna vazia lá no arquivo `.csv`. E também, a última linha é a linha de total, o que pode ser ruim para o autoschema do BigQuery, que é o que utilizaremos para esse projeto.

Para evitar problemas futuros, vamos remover essa coluna e também essa linha.

Aproveitando que já estamos alterando o arquivo base, vamos adicionar a coluna `dt_insert`.

In [76]:
df = df.drop(columns=["Unnamed: 3"]).drop(df.index[-1])
df['dt_insert'] = pendulum.today('utc').to_iso8601_string()

In [80]:
df.head()

,Fonte de Recursos,Despesa,Liquidado,dt_insert
0,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUBLICO,"79.760.504,67",2024-09-07T00:00:00Z
1,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900124 - COMPLEMENTACAO DE APOSENTADORIA,"1.850.834.374,26",2024-09-07T00:00:00Z
2,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900125 - COMPLEMENTACAO DE APOSENTADORIA - 13ºSALARIO,"124.536.202,87",2024-09-07T00:00:00Z
3,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900325 - COMPLEMENTACAO DE PENSAO,"10.948.297,00",2024-09-07T00:00:00Z
4,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900326 - COMPLEMENTACAO DE PENSAO - 13º SALARIO,"521.632,30",2024-09-07T00:00:00Z


## E agora?

Aqui já temos o dado pronto para ir para a camada Raw.

Podemos passar para os dados da tabela de Receitas.

Developed by

```shell
             ___         __  __              
            /   |  _____/ /_/ /_  __  _______
           / /| | / ___/ __/ __ \/ / / / ___/
          / ___ |/ /  / /_/ / / / /_/ / /    
         /_/  |_/_/   \__/_/ /_/\__,_/_/     
```